##Zadanie 1.
Zaimplementuj w środowisku MPI obliczanie równoległe liczby PI z wzoru Leibniz-a.
Proces 0, to proces Root, który zarządza obliczeniami:
- wysyła do procesów roboczych Slave, indeksy wyrazów szeregu poczatek_local i koniec_local, 
z których należy obliczyć wyniki cząstkowe
- zbiera wyniki cząstkowe z procesów i wyświetla wynik przybliżenia PI

In [20]:
%%sh
cat > pi-mpi.c <<EOF
#include <stdlib.h>
#include <stdio.h>
#include <mpi.h>
#include <time.h>
#include <sys/time.h>

// Leibniz 

double potega(double podstawa , int wykladnik ){
  	double wynik=1.0;
  for (int i = 1; i <= wykladnik; i++)
    {
        wynik*=podstawa;
    }


  return wynik ; 
}

int main(int argc, char *argv[])
{
    //tworzenie zmiennych globalnych
    int i,np,lp;
    int tag=50;
    MPI_Status status;
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &np);
    MPI_Comm_size(MPI_COMM_WORLD, &lp);
    
    //jesli proces ma numer 0 to:
    if(np ==0)
    {
      //tworzenie zmiennych lokalnych
      double pole = 0 ,s=0;
      printf("\n Obliczenie liczny pi ze wzoru leibniza");
   
      //odbieranie oraz sumowanie wynikow z wszystkich procesow

      for(i=1; i<lp; i++)
      {
        MPI_Recv(&s, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD, &status);
        pole +=s;
      }

      //wypisanie wartosci pola
      printf("\nwartosc liczby pi ze wzoru Leibnitza = %1f\n", 4 *  pole);
    }

    if(np != 0)
    {
      //tworzenie zmiennych lokalnych
      double s1 = potega( -1 , np - 1   )   / ( 2 *  np - 1  )   ; 
      printf("%f :  " , s1);
      // s1 = 0;
      //wysylanie liczby 
      MPI_Send(&s1, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD);
    }
    MPI_Finalize();
}
EOF
mpicc pi-mpi.c && mpirun -n 40 --allow-run-as-root a.out


 Obliczenie liczny pi ze wzoru leibniza
wartosc liczby pi ze wzoru Leibnitza = 3.167229
-0.018182 :  0.012987 :  -0.090909 :  -0.014085 :  0.034483 :  -0.052632 :  0.200000 :  -0.025641 :  -0.016949 :  -0.019608 :  0.047619 :  -0.023256 :  -0.015873 :  0.076923 :  0.040000 :  0.013699 :  -0.333333 :  0.111111 :  -0.043478 :  -0.032258 :  0.030303 :  0.024390 :  0.022222 :  -0.021277 :  0.018868 :  0.016393 :  -0.014925 :  -0.013333 :  -0.142857 :  0.027027 :  0.017544 :  0.015385 :  1.000000 :  -0.066667 :  0.014493 :  0.058824 :  -0.037037 :  -0.028571 :  0.020408 :  

##Zadanie 2
Zaimplementuj w środowisku MPI wyznaczenie numerycznej wartość całki y=f(x) (postać funkcji 
wybierasz sam) w przedziale <a,b> przy pomocy N trapezów.
Proces 0, to proces Root, który zarządza obliczeniami:
- wysyła do procesów roboczych Slave początek a_local i koniec b_local lokalnego przedziału 
całkowania dla danego procesu oraz liczbę N_local trapezów, z których należy policzyć całkę
- zbiera wyniki cząstkowe z procesów, i wyświetla wynik całki

In [13]:
%%sh 
cat > pi-mpi.c << EOF
#include <mpi.h>
#include <stdio.h>
double funkcja(double x) {
return x; 
}
int main(int argc, char *argv[]) {
int i,np,lp;
int tag=50;
MPI_Status status;
MPI_Init(&argc, &argv);
MPI_Comm_rank(MPI_COMM_WORLD, &np);
MPI_Comm_size(MPI_COMM_WORLD, &lp);
//jeśli proces ma numer 0 to:
if (np == 0) {
//Tworzenie zmiennych lokalnych
double h,xp,xk,pole,s=0;
printf("\nObliczanie calki oznaczonej metoda trapezu dla funkcji f(x)=x");
//Poczatek przedzialu calkowania xp 
xp = 1.0;
//Koniec przedzialu calkowania xk 
xk = 4.0;
// obliczanie wartości h
h=(xk-xp)/(double)lp;
// wysylanie wartosci ci xp i xk oraz h do poszczegolnych procesow
for(i=0;i<lp;i++)
{
MPI_Send(&xp, 1, MPI_DOUBLE, i,tag, MPI_COMM_WORLD);
MPI_Send(&xk, 1, MPI_DOUBLE, i,tag, MPI_COMM_WORLD);
MPI_Send(&h, 1, MPI_DOUBLE, i,tag, MPI_COMM_WORLD);
}
// odbieranie oraz sumowanie wyników z wszystkich procesów 
for(i=0;i<lp;i++)
{
MPI_Recv(&s, 1, MPI_DOUBLE, i, tag,MPI_COMM_WORLD, &status);
pole+=s;
printf("Pole trapezu wynosi: %1f\n"  , s) ;

}
// wypisanie wartości pola
printf("\nPole pod calka wynosi = %lf\n",pole); 
}
else
{
//tworzenie zmiennych lokalnych
double s,h,xp,xk;
double np2=np;
// odbieranie wartości od procesu 0 
MPI_Recv(&xp, 1, MPI_DOUBLE, 0, tag,MPI_COMM_WORLD, &status);
MPI_Recv(&xk, 1, MPI_DOUBLE, 0, tag,MPI_COMM_WORLD, &status);
MPI_Recv(&h, 1, MPI_DOUBLE, 0, tag,MPI_COMM_WORLD, &status);
// obliczanie pola trapezu dla podanej funkcji
s=( funkcja(  xp  +  (np - 1 )  * h)  + funkcja(  xp  +  np * h)  ) * 0.5 * h; 
//wysyłanie pola trapezu do procesu 0
MPI_Send(&s, 1, MPI_DOUBLE, 0,tag, MPI_COMM_WORLD); 
}
MPI_Finalize();
}
EOF
mpicc pi-mpi.c && mpirun -n 8 --allow-run-as-root a.out



Obliczanie calki oznaczonej metoda trapezu dla funkcji f(x)=xPole trapezu wynosi: 1.000000
Pole trapezu wynosi: 0.445312
Pole trapezu wynosi: 0.585938
Pole trapezu wynosi: 0.726562
Pole trapezu wynosi: 0.867188
Pole trapezu wynosi: 1.007812
Pole trapezu wynosi: 1.148438
Pole trapezu wynosi: 1.289062

Pole pod calka wynosi = 7.070312


##Zadanie 3
Zaimplementuj program mnożenia macierzy A przez wektor B
Proces 0, to proces Root, który zarządza obliczeniami:
- inicjuje wartości macierzy A i wektora B 
- wysyła do procesów roboczych Slave fragmenty macierzy A i wektora B 
- zbiera wyniki cząstkowe z procesów, i wyświetla wynik całki

In [ ]:
%%sh
cat > pi-mpi.c << EOF

#include <stdlib.h>
#include <stdio.h>
#include <mpi.h>
#include <time.h>
#include <sys/time.h>
#define NELTS 4 /* number of elements in the vector */
#define MASTER 0 /* id of the first process */
#define FROM_MASTER 1 /* a message type */
#define FROM_WORKER 2 /* a message type */
#define N 4





MPI_Status status;
int main(int argc, char *argv[]) {
  double v[NELTS], /* the vector to be summed */
  sum, /* the overall vector sum */
  ptSum; /* a partial sum received from a worker */
  int numProcs, /* number of processes in virtual machine */
  numWorkers,   /* number of worker processes */
  rank,          /* our rank */
  source,      /* rank of message source */
  dest,      /* rank of message destination */
  mtype,     /* message type */
  aveSz,     /* average partition size */
  extra,     /* number of left over elements */
  offset,    /* offset of partition in v */
  i,        /* simple counter */
  count;   /* the actual number of elements in a partition */


//Liczba wierszy i kolumn w macierzach A, B, C
double A[N][N],B[N][N],C[N][N];
int processCount, processId, slaveTaskCount ;

// MPI 
MPI_Init(&argc, &argv);
MPI_Comm_rank(MPI_COMM_WORLD, &rank);
MPI_Comm_size(MPI_COMM_WORLD, &numProcs);
numWorkers = numProcs-1;


/******* master process ***********/
if (rank == MASTER) {
  // Initialize the vector
  for (i=0; i<NELTS; i++)v[i]= 1;
  
  
  // send vector partitions to the workers
  aveSz = NELTS/numWorkers;
  extra = NELTS%numWorkers;
  offset = 0;
  mtype = FROM_MASTER;
  
  for (dest=1; dest<=numWorkers; dest++) { 
    // start at 1 not 0 (no master)
    count = (dest <= extra) ? aveSz+1 : aveSz;
    MPI_Send(&count,1,MPI_INT,dest,mtype, MPI_COMM_WORLD);
    MPI_Send(&v[offset],count,MPI_DOUBLE,dest,mtype,MPI_COMM_WORLD);
    printf("Master sent elements %d to %d to rank %d.\n", offset,offset+count,dest);
    offset+=count;
  }


    // Inicjalizacja  Maceierze Dwuwymiarowej 
    // Inicjalizacja macierzy A i B


      double start = MPI_Wtime();
      srand ( time(NULL) );
      
      for (int i = 0; i<N; i++) {
        for (int j = 0; j<N; j++) {
        A[i][j]= rand()%10;
        B[i][j]= rand()%10;
      }}
      
      //Wypisanie zawartości macierzy A i B
      printf("\n Mnozenie macierzy za pomoca MPI \n");
      printf("\nMacierz A\n\n");
      
      for (int i = 0; i<N; i++) {
        for (int j = 0; j<N; j++) {
        printf("%.0f\t", A[i][j]);
      }
    printf("\n");
  }
  printf("\nMacierz B\n\n");
  for (int i = 0; i<N; i++) {
  for (int j = 0; j<N; j++) {
  printf("%.0f\t", B[i][j]);
  }
  }


/* wait for results from workers and sum them */
sum=0;
mtype = FROM_WORKER;


for (i=1; i<=numWorkers; i++) { 
  // start at 1 not 0 (no master)
  source = i;
  MPI_Recv(&ptSum,1,MPI_DOUBLE,source,mtype,MPI_COMM_WORLD, &status);
  sum+=ptSum;
}
  printf("Vector sum is %lf.\n", sum);
}

 // end master
/************ worker processes *************/
if (rank > MASTER) {
// receive out partition from the master
mtype = FROM_MASTER;
source = MASTER;
MPI_Recv(&count,1,MPI_INT,source,mtype,MPI_COMM_WORLD,&status);
MPI_Recv(&v,count,MPI_DOUBLE,source,mtype,MPI_COMM_WORLD,&status);
// compute our partial sum
sum=0;
//for (i=0;i<count;i++)
 // sum+=v[i];  




 for(int o=0;o<count;o++)
 {
 	for(int p=0;p<count;p++)
 	{
 		sum =+  v[o] * A[o][p];
 	//	printf("[%d]=%.1f\t", p, s[p]);
 	}
 }
  
/***/ printf("Slave %d is sending partial sum %lf to master.\n",rank,sum);
// send our partial sum to the master
mtype = FROM_WORKER;
MPI_Send(&sum,1,MPI_DOUBLE,MASTER,mtype,MPI_COMM_WORLD);
} // end worker
MPI_Finalize();
}
EOF
mpicc pi-mpi.c && mpirun -n 5 --allow-run-as-root a.out